# Pull MIP table files; strip out extraneous variables
<div style="text-align: right">
<p>
    <img src="https://pcmdi.github.io/assets/PCMDI/199x65px-PCMDI-Logo-Text-rectangle.png"
         width="91"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="Program for Climate Model Diagnosis and Intercomparison"
         alt="Program for Climate Model Diagnosis and Intercomparison"
    >&nbsp;
    <img src="https://pcmdi.github.io/assets/LLNL/212px-LLNLiconPMS286-WHITEBACKGROUND.png"
         width="30"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="Lawrence Livermore National Laboratory"
         alt="Lawrence Livermore National Laboratory"
    >&nbsp;
    <img src="https://pcmdi.github.io/assets/DOE/459x127px-DOE-Logo_Color_TextOnly.png"
         width="108"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="United States Department of Energy"
         alt="United States Department of Energy"
    >
</p>
</div>

**Summary**

This file pulls a CMIP6Plus/CMOR3.9.0-era MIP table files, strips out
extraneous variables and saves the files for local use

**Authors**

Paul J. Durack ([durack1](https://github.com/durack1); [PCMDI](https://pcmdi.llnl.gov/), [Lawrence Livermore National Laboratory](https://www.llnl.gov/))

**Notes**

PJD 14 Feb 2025 - initiated<br>
PJD 14 Feb 2025 - updated with coordinate, formula_terms and grids<br>
TODO: Add "Header" with provenance info to coordinate, formula_terms and grid tables

**Links**

### imports

In [1]:
%%time
import datetime
import json
import os
import requests

CPU times: user 29.2 ms, sys: 12.2 ms, total: 41.4 ms
Wall time: 57.1 ms


### set table, coordinate, formula_terms and grids URLs and pull files

In [2]:
%%time
urls = {
    "APday": "https://raw.githubusercontent.com/PCMDI/mip-cmor-tables/refs/heads/main/Tables/MIP_APday.json",
    "coordinate": "https://raw.githubusercontent.com/PCMDI/input4mips-cmor-tables/refs/heads/master/Tables/input4MIPs_coordinate.json",
    "formula_terms": "https://raw.githubusercontent.com/PCMDI/input4mips-cmor-tables/refs/heads/master/Tables/input4MIPs_formula_terms.json",
    "grids": "https://raw.githubusercontent.com/PCMDI/input4mips-cmor-tables/refs/heads/master/Tables/input4MIPs_coordinate.json",
}

# loop through urls
keys = urls.keys()
for count, key in enumerate(keys):
    print(count, key)
    url = urls[key]
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4/5xx)
        vars()[key] = json.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decode failed: {e}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")

0 APday
1 coordinate
2 formula_terms
3 grids
CPU times: user 34.6 ms, sys: 14.1 ms, total: 48.7 ms
Wall time: 20.7 s


### APday - trim out redundant variables

In [3]:
keyList = list(APday["variable_entry"].keys())
keepKeys = ["pr", "tasmax", "tasmin"]
for count, key in enumerate(keyList):
    if key not in keepKeys:
        APday["variable_entry"].pop(key)

### check remaining table entries

In [4]:
APday["variable_entry"].keys()

dict_keys(['pr', 'tasmax', 'tasmin'])

### update header

In [5]:
APday["Header"]

{'Conventions': 'CF-1.7 CMIP-6.5',
 'approx_interval': 1.0,
 'checksum': '',
 'cmor_version': '3.8.0',
 'data_specs_version': '6.5.0.0',
 'generic_levels': '',
 'int_missing_value': '-999',
 'missing_value': '1e20',
 'product': 'model-output',
 'table_date': '2024-02-28',
 'table_id': 'APday'}

In [6]:
%%time
# get current commit hash
url = "https://api.github.com/repos/PCMDI/mip-cmor-tables/commits/main"
response = requests.get(url)
js = json.loads(response.text)
mipTHash = js["sha"]
url = "https://api.github.com/repos/PCMDI/input4MIPs-cmor-tables/commits/master"
response = requests.get(url)
js = json.loads(response.text)
inp4Hash = js["sha"]
print("mipTHash:", mipTHash)
print("inp4Hash:", inp4Hash)

mipTHash: f42386929a0057ed15e66a3bac045b8c00d33c0f
inp4Hash: e4fa82f330125fbdd78fa8315225829eabffee70
CPU times: user 17.1 ms, sys: 4.46 ms, total: 21.5 ms
Wall time: 10.7 s


In [ ]:
APday["Header"]["cmor_version"] = "3.9.0"
APday["Header"]["data_specs_version"] = " ".join(
    ["6.5.0.0; mip-cmor-tables;", mipTHash]
)
APday["Header"]["table_date"] = datetime.datetime.now().strftime("%Y-%m-%d")
APday["Header"]["product"] = "DRCDP"
APday["Header"].pop("checksum")  # remove invalid entry, CMOR 3.9.0
APday["Header"]

{'Conventions': 'CF-1.7 CMIP-6.5',
 'approx_interval': 1.0,
 'cmor_version': '3.9.0',
 'data_specs_version': '6.5.0.0; mip-cmor-tables; f42386929a0057ed15e66a3bac045b8c00d33c0f',
 'generic_levels': '',
 'int_missing_value': '-999',
 'missing_value': '1e20',
 'product': 'DRCDP',
 'table_date': '2025-02-14',
 'table_id': 'APday'}

### build Header entry for coordinate, formula_terms and grids

In [8]:
Header = {}
# Header["Conventions"] = "CF-1.7 CMIP-6.5"
# Header["approx_interval"] = 1.0
Header["cmor_version"] = "3.9.0"
Header["data_specs_version"] = " ".join(["6.5.0.0; input4MIPs-cmor-tables;", inp4Hash])
# Header["generic_levels"] = ''
# Header["int_missing_value"] = '-999'
# Header["missing_value"] = '1e20'
Header["product"] = "DRCDP"
Header["table_date"] = datetime.datetime.now().strftime("%Y-%m-%d")
# Header["table_id"] = 'keyFill'

### write all files out to Tables subdir

In [9]:
tableName = "../Tables/DRCDP_APday.json"
files = ["APday", "coordinate", "formula_terms", "grids"]
for count, name in enumerate(files):
    print(count, name)
    dic = eval(name)
    # add Header
    if name not in ["APday", "coordinate", "formula_terms"]:
        # exclude Header rewrite for APday, Header add for coordinate/formula_terms as changes CMOR 3.9.0 behaviours
        dic["Header"] = Header
    # write file
    outFile = "".join(["../Tables/DRCDP_", name, ".json"])
    with open(outFile, "w") as f:
        json.dump(
            dic, f, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":")
        )

0 APday
1 coordinate
2 formula_terms
3 grids
